# Build a Hidden Markov Model Stock Excess Growth Rate for One Minute Aggregates
This advanced example will build upon the previous example and will give students practice building an Observable Markov Model (OMM) of the excess growth rate of a ticker `SPY` using one-minute aggregate data (instead of daily data). The excess growth:
$$
\begin{equation*}
R_{ij} = \left(\frac{1}{\Delta{t}}\right)\cdot\ln\left(\frac{S_{i,j}}{S_{i,j-1}}\right) - \bar{r}_{f}
\end{equation*}
$$
is a scaled return, assuming continuous compounding. The quantity $R_{ij}$ denotes the excess growth rate of equity $i$ at time $j$ (units: 1/year), $\Delta{t}$ denotes the time-step between $j-1\rightarrow{j}$ (units: years), $S_{i,\star}$ denotes the share price of equity $i$ at time $\star$, and $\bar{r}_{f}$ denotes the annualized risk free rate.

Describe the minute-to-minute variation of the excess growth using a fully observable Markov model $\mathcal{M}$ represented by the tuple $\mathcal{M} = (\mathcal{S},\mathcal{O},\mathbf{P},\mathbf{E})$; $\mathcal{S}$ is the set of hidden states, $\mathcal{O}$ is the set of observable states, $\mathbf{T}$ is the transition matrix and $\mathbf{E}$ is the emission matrix. Because we are fully observable, the emission matrix $\mathbf{E} = \mathbf{I}$, where $\mathbf{I}$ is the identity matrix.

## Learning objectives
* __Task 1__: Construct the states $\mathcal{S}$, the emission matrix $\mathbf{E}$ and transition matrix $\hat{\mathbf{T}}$.
    * Estimate the transition matrix $\hat{\mathbf{T}}$ from market data
* __Task 2__: Simulate the Hidden Markov Model (HMM) for approximately one trading year in one-minute aggregates
    * Generate the stationary distribution from the estimated $\hat{\mathbf{T}}$ matrix
    * Implement the `MARKOV-SIMULATION` pseudo code from the lecture to generate hypothetical return sequences

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [1]:
include("Include.jl");

## Prerequisites: Constants and process the one-minute aggregate share price data
Before we set up the Markov model, we set some constant values that we'll use throughout the rest of the notebook. Then, we'll load and process the 1-minute aggregate share price data for `SPY.`

### Constants

In [2]:
risk_free_rate = 0.0421; # 17-Jun-2024 10-year treasury
Δt = (1/390)*(1/252); # 1-min aggregate trading periods in units of years
ticker = "SPY";

### Load and process `SPY` data
The `SPY` data used in this project (the entire 2023 trading year) was downloaded from [oneminutedata.com](https://oneminutedata.com). The `SPY` dataset, which is stored as a [comma-separated value (CSV) file](https://en.wikipedia.org/wiki/Comma-separated_values), is loaded using the [read method exported by the CSV.jl package](https://github.com/JuliaData/CSV.jl) as a [DataFrame instance](https://github.com/JuliaData/DataFrames.jl). We save the `SPY` data in the `share_price_data_df::DataFrame` variable:

In [3]:
share_price_data_df = CSV.read(joinpath(_PATH_TO_DATA, "SPY-OHLC-1-min-aggregate-2023.csv"), DataFrame)

Row,date,open,high,low,close,volume
,String31,Float64,Float64,Float64,Float64,Int64
1,2023-01-03 09:30:00,384.37,384.46,383.97,384.13,575276
2,2023-01-03 09:31:00,384.13,384.13,383.59,383.96,293800
3,2023-01-03 09:32:00,383.99,384.42,383.99,384.154,289329
4,2023-01-03 09:33:00,384.16,385.08,384.15,385.07,300648
5,2023-01-03 09:34:00,385.06,385.12,384.47,384.51,246156
6,2023-01-03 09:35:00,384.54,384.99,384.53,384.755,230167
7,2023-01-03 09:36:00,384.75,384.78,384.325,384.53,119297
8,2023-01-03 09:37:00,384.54,384.855,384.375,384.741,129655
9,2023-01-03 09:38:00,384.71,384.89,384.61,384.79,120879


Next, compute the [volume weighted average price](https://en.wikipedia.org/wiki/Volume-weighted_average_price#:~:text=In%20finance%2C%20volume%2Dweighted%20average,trading%20price%20for%20the%20period.) using the [vwap function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.vwap) for the `share_price_data_df::DataFrame.` We then compute the growth rate, i.e., time-scaled annualized return using [the `log_growth_matrix(...)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix), where $\Delta{t}$ denotes the trading period, i.e., 1-minute in units of years. We save growth rate data in the `Rᵢ::Array{Float64,1}` array, where each row holds the growth rate for that trading time period:

In [4]:
Rᵢ = vwap(share_price_data_df)|> data -> log_growth_matrix(data, Δt = Δt)

96879-element Vector{Float64}:
 -25.370811956266607
   6.433212218622656
  34.47544744550597
  16.70915720871667
  13.550730179623784
   2.661297195665073
   4.2649203762947865
   4.967265992331227
  16.829867852257006
  24.785890818294575
  40.51341066698778
  33.242673507082735
  20.23424653772622
   ⋮
   0.3406377163070191
   0.3162875911374961
   0.6446669742260449
   0.5396951870740953
   0.3922352799593621
   0.2880388669883392
   0.7142093136693322
   0.718297201272962
   0.4765247924607921
   0.6185536977465691
   1.011400876667081
   5.916606670551687

## Task 1: Construct the states $\mathcal{S}$, the emission matrix $\mathbf{E}$ and transition matrix $\hat{\mathbf{T}}$
First, consider the states $\mathcal{S}$. Suppose we put a label (and number) the excess return values, ranging from `super bad = 1,` $\dots$,` unchanged,` $\dots$,` super good = N,` where if $R\ll{0}$, then we are in the `super bad = 1,` state or $R\gg{0}$ we are in the `super good = N` state (or we are someplace in between). 
* __Idea__: Use the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) computed from the observed return series $R_{i,1}, \dots, R_{i,n}$ to partition the actual (historical) excess growth rates into one of a fixed number of categories. Once we have the categories, compute the probability that category $i$ on the day $k$ is followed by category $j$ on the day $k+1$. These values are entries in the state transition matrix $\hat{\mathbf{T}}$.
* To start, specify a value for the  `number_of_states` variable, where the `number_of_states` controls how many categories we are using when splitting up the excess return time series. We then set the `states` vector, which holds the states (numbered from `1`$\rightarrow$`number_of_states`). 

In [5]:
number_of_states = 2000; # TODO: specify a value here
states = range(1,stop=number_of_states) |> collect;

The `states` are hidden from the observer. Next, we set up the emissions matrix $\mathbf{E}$. For this example, because the states are __fully observable__, i.e., we can see the states directly,  the emission matrix $\mathbf{E}$ is the identity matrix $\mathbf{I}$:

In [6]:
E = diagm(ones(number_of_states));

### TODO: Estimate the transition matrix $\hat{\mathbf{T}}$ from market data
To estimate the transition matrix $\hat{\mathbf{T}}$, we'll estimate the transition probabilities from the excess return data calculated in the `Prerequisites` section and saved in the `Rᵢ` variable:
* Split the data into two blocks: the first (which we'll call the `in_sample_dataset`) will be used to estimate the elements of the matrix $\hat{\mathbf{T}}$, while the second (which we'll call `out_of_sample_dataset`) will be used for testing purposes (later). The fraction of data partitioned into the training dataset is controlled by the `split_fraction` variable.

Set a value for the `split_fraction` variable:

In [7]:
split_fraction = 0.90; # TODO: specify a value here
insample_end_index = round(split_fraction*length(Rᵢ), digits=0) |> Int
in_sample_dataset = Rᵢ[1:insample_end_index]
out_of_sample_dataset = Rᵢ[(insample_end_index+1):end];

Next, we need to model the return data distribution to compute the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function). The excess return distribution is a random variable that follows some [probability distribution function](https://en.wikipedia.org/wiki/Probability_distribution). We don't know what that distribution function is, but for now, we assume the excess returns follow a [Laplace distribution](https://en.wikipedia.org/wiki/Laplace_distribution)
* We use the [fit_mle function](https://juliastats.org/Distributions.jl/stable/fit/#Distributions.fit_mle-Tuple{Any,%20Any}) exported by the [Distributions.jl package](https://github.com/JuliaStats/Distributions.jl) to fit a [Laplace distribution](https://en.wikipedia.org/wiki/Laplace_distribution) to our our data. We fit the distribution to the full dataset `Rₘ` and save the distribution in the `d` variable.

In [25]:
d = fit_mle(InverseGaussian, Rᵢ); # use the *full* data set to establish the cutoff's

Next, we generate the percentile cutoffs that we use to establish the bounds that correspond to each category of return, i.e., `super bad` or `super good`, etc.

In [26]:
percentage_cutoff = range(0.0,stop=1.0,length=(number_of_states+1)) |> collect;

Now that we have the cutoffs, compute the lower and upper bound for each potentiual category. To do this, we'll use the [quantile function](https://juliastats.org/Distributions.jl/stable/univariate/#Statistics.quantile-Tuple{UnivariateDistribution,%20Real}) exported by the [Distributions.jl package](https://github.com/JuliaStats/Distributions.jl). For a given `0 ≤ q ≤ 1`, `quantile(d, q)` is the smallest value `x` 
for which `cdf(d, x) ≥ q`.

In [27]:
bounds = Array{Float64,2}(undef, number_of_states, 3)
for s ∈ states
    bounds[s,1] = quantile(d,percentage_cutoff[s])
    bounds[s,2] = quantile(d,percentage_cutoff[s+1])
    bounds[s,3] = s
end
bounds

2000×3 Matrix{Float64}:
 0.0         0.0147874     1.0
 0.0147874   0.0161118     2.0
 0.0161118   0.0170015     3.0
 0.0170015   0.0176942     4.0
 0.0176942   0.0182713     5.0
 0.0182713   0.0187713     6.0
 0.0187713   0.0192157     7.0
 0.0192157   0.019618      8.0
 0.019618    0.019987      9.0
 0.019987    0.0203289    10.0
 0.0203289   0.0206484    11.0
 0.0206484   0.020949     12.0
 0.020949    0.0212332    13.0
 ⋮                      
 0.378739    0.384894   1989.0
 0.384894    0.391652   1990.0
 0.391652    0.39914    1991.0
 0.39914     0.407533   1992.0
 0.407533    0.417076   1993.0
 0.417076    0.428128   1994.0
 0.428128    0.441248   1995.0
 0.441248    0.457375   1996.0
 0.457375    0.478274   1997.0
 0.478274    0.507927   1998.0
 0.507927    0.559112   1999.0
 0.559112   Inf         2000.0

Now that we have the category bounds, let's take the excess return data and determine which state an excess return observation corresponds to. For each sample in the `in_sample_dataset`:
* Classify the sample value into one of the possible categories. Let `state = 1` equal the worst return, and `state = number_of_states` equal the best return. Save these results in the `encoded_in_sample` array:

In [28]:
encoded_in_sample = Array{Int64,1}();
for i ∈ eachindex(in_sample_dataset)
    value = in_sample_dataset[i];

    class_index = 1;
    for s ∈ states
        if (bounds[s,1] ≤ value && value < bounds[s,2])
            class_index = s;
            break;
        end
    end
    push!(encoded_in_sample, class_index);
end
encoded_in_sample;

In the matrix $\mathbf{T}$ compute the `counts` for transition from state `i` to state `j`:

In [29]:
T = zeros(number_of_states, number_of_states)
for i ∈ 2:insample_end_index
    start_index = encoded_in_sample[i-1];
    stop_index = encoded_in_sample[i];
    T[start_index,stop_index] += 1;
end
T

2000×2000 Matrix{Float64}:
 14833.0  142.0  86.0  58.0  46.0  42.0  …   0.0   0.0    0.0    0.0     2.0
   142.0   32.0  25.0  15.0  11.0   9.0      0.0   0.0    0.0    0.0     0.0
   103.0   21.0  18.0  11.0  12.0   7.0      0.0   0.0    0.0    0.0     0.0
    65.0   15.0  13.0  14.0   6.0   6.0      0.0   0.0    0.0    0.0     0.0
    59.0   11.0  14.0   3.0   8.0   7.0      0.0   0.0    0.0    0.0     0.0
    37.0   14.0   6.0  10.0   4.0   5.0  …   0.0   0.0    0.0    0.0     0.0
    44.0   12.0  11.0   4.0  11.0   6.0      0.0   0.0    0.0    0.0     0.0
    33.0   14.0  12.0   5.0   5.0   6.0      0.0   0.0    0.0    0.0     0.0
    27.0    6.0   4.0   3.0   7.0   0.0      0.0   0.0    0.0    0.0     0.0
    15.0   10.0   6.0   6.0   5.0   4.0      0.0   0.0    0.0    0.0     0.0
    28.0    8.0   9.0   0.0   0.0   4.0  …   0.0   0.0    0.0    0.0     1.0
    13.0    5.0   7.0   2.0   1.0   2.0      0.0   0.0    0.0    0.0     0.0
    16.0    5.0   0.0   3.0   3.0   1.0      0.0 

From the `counts` matrix $\mathbf{T}$, compute the transtion probability matrix $\hat{\mathbf{T}}$:

In [30]:
T̂ = zeros(number_of_states, number_of_states)
for row ∈ states
    Z = sum(T[row,:]);
    for col ∈ states
        T̂[row,col] = (1/Z)*T[row,col]
    end
end
T̂

2000×2000 Matrix{Float64}:
 0.928804     0.00889167  0.0053851  …  0.0        0.0        0.000125235
 0.276803     0.0623782   0.0487329     0.0        0.0        0.0
 0.237327     0.0483871   0.0414747     0.0        0.0        0.0
 0.223368     0.0515464   0.0446735     0.0        0.0        0.0
 0.208481     0.0388693   0.04947       0.0        0.0        0.0
 0.152263     0.0576132   0.0246914  …  0.0        0.0        0.0
 0.169231     0.0461538   0.0423077     0.0        0.0        0.0
 0.157143     0.0666667   0.0571429     0.0        0.0        0.0
 0.135        0.03        0.02          0.0        0.0        0.0
 0.0815217    0.0543478   0.0326087     0.0        0.0        0.0
 0.148936     0.0425532   0.0478723  …  0.0        0.0        0.00531915
 0.081761     0.0314465   0.0440252     0.0        0.0        0.0
 0.112676     0.0352113   0.0           0.0        0.0        0.0
 ⋮                                   ⋱                        
 0.0          0.0         0.0        

## Task 2: Simulate the Hidden Markov Model (HMM)
To do the simulation, we first build a [`MyHiddenMarkovModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/markov/#VLQuantitativeFinancePackage.MyHiddenMarkovModel), which holds the data for our Markov model. We use a [`build(...)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/markov/#VLQuantitativeFinancePackage.build-Tuple{Type{MyHiddenMarkovModel},%20NamedTuple}), which takes information about the `states,` the estimated transition matrix $\hat{\mathbf{T}}$, and the emission matrix $\mathbf{E}$ and returns a [`MyHiddenMarkovModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/markov/#VLQuantitativeFinancePackage.MyHiddenMarkovModel), which we save in the `model` variable.

In [31]:
model = build(MyHiddenMarkovModel, (
    states = states,
    T = T̂, 
    E = E
));

### TODO: Generate the stationary distribution from the estimated $\hat{\mathbf{T}}$ matrix
Generate the stationary distribution for the estimated transition matrix $\hat{\mathbf{T}}$ and use it to construct a [Categorical distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Categorical) representing the stationary distrubution, save the [Categorical distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Categorical) in the `π̄`-variable:

In [32]:
power_value = 10000;
π̄ = (T̂^power_value) |> tmp -> Categorical(tmp[1,:]); # TODO: compute the stationary distribution (approx value is ok)

#### Check: Are you really at the stationary distribution?
To compute the stationary distribution $\bar{\pi}$, we raise the transition matrix $\hat{\mathbf{T}}$ to the power of `power_value.` However, we don't know the value of `power_value` _a priori_. 
* We can test different values for `power_value` using the fact that stationary distribution is a rank one matrix. Thus, we can estimate the minimum value of `power_value` such that `T̂^power_value` has `rank = 1` and check this condition [using the Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert).

In [33]:
@assert ((T̂^power_value) |> rank ) == 1

### TODO: Implement the `MARKOV-SIMULATION` pseudo code to generate hypothetical return sequences
*Generate 'number_of_paths' example sequences, each containing 'number_of_steps' days. These variables determine the length and number of our hypothetical return sequences. Assume each path starts from a draw from the stationary distribution `π̄.`
* Save the simulated return sequences in the `archive::Array{Int64,2}(undef, number_of_steps, number_of_paths)` array, where the `row` index corresponds to a path, and the `col` index corresponds to a day.
* We have implemented some shortcut logic to speed up the implementation. To evaluate the Markov model for a `number_of_steps,` issue the command `model(start_state, number_of_steps).` This will compute a chain with `number_of_steps` entries starting as `start_state` and return the simulated sequence as an `array.` 

In [34]:
number_of_paths = 100;
number_of_steps = insample_end_index; # average number of trading days per year
encoded_archive = Array{Int64,2}(undef, number_of_steps, number_of_paths);
for i ∈ 1:number_of_paths
    start_state = rand(π̄);
    tmp = model(start_state, number_of_steps)
    for j ∈ 1:number_of_steps
        encoded_archive[j,i] = tmp[j]
    end
end
encoded_archive

87191×100 Matrix{Int64}:
    1     1  1071     1  242   797  …  1328   797   612  1752   376   587
    1     1   878     1   40   718     1706  1296    72  1905   244   157
    1     1   963     1  113   649     1135   604    99  1219   964   391
    1     1  1188     1   58  1368     1527   485    83  1152  1648  1541
    1     1  1020     1   94  1616      727   381   252  1219  1903  1861
    4     2  1393     1   76  1713  …   920    38    48  1742  1493  1935
  141    89  1890     1   26  1363      486    36    73  1977  1038  1973
  658   510  1905     1  133  1018     1157    85   155  1826  1632  1948
  563    40  1254     1  184  1168     1012  1108   773  1917  1731  1521
  286   240  1336     1  177   821      477  1730   580  1946  1581  1668
 1088   291   306     1  224  1455  …   401  1107   961  1757   966  1879
 1736   369    17     1    7  1305      140  1301    39  1634   373  1865
 1902   812   110     1  163  1528      198  1644   512  1372   316  1709
    ⋮        

In [35]:
actual_sample_bounds = copy(bounds);
actual_sample_bounds[1,1] = minimum(in_sample_dataset);
actual_sample_bounds[end,2] = maximum(in_sample_dataset)
actual_sample_bounds

2000×3 Matrix{Float64}:
 -25.3708      0.0147874     1.0
   0.0147874   0.0161118     2.0
   0.0161118   0.0170015     3.0
   0.0170015   0.0176942     4.0
   0.0176942   0.0182713     5.0
   0.0182713   0.0187713     6.0
   0.0187713   0.0192157     7.0
   0.0192157   0.019618      8.0
   0.019618    0.019987      9.0
   0.019987    0.0203289    10.0
   0.0203289   0.0206484    11.0
   0.0206484   0.020949     12.0
   0.020949    0.0212332    13.0
   ⋮                      
   0.378739    0.384894   1989.0
   0.384894    0.391652   1990.0
   0.391652    0.39914    1991.0
   0.39914     0.407533   1992.0
   0.407533    0.417076   1993.0
   0.417076    0.428128   1994.0
   0.428128    0.441248   1995.0
   0.441248    0.457375   1996.0
   0.457375    0.478274   1997.0
   0.478274    0.507927   1998.0
   0.507927    0.559112   1999.0
   0.559112   40.5134     2000.0

### Decode operation
To turn the state $s\in\mathcal{S}$ back into an excess return value, we need to __decode__ the state. To do this, let's construct a [Normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) describing the observed return values associated with each state in the `encoded_in_sample` dataset.
* We collect the observed excess return samples associated with a particular state $s$, store them in a `tmp` array, and use [Maximum Likelihood Estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) to estimate a [Normal distribution exported from the Distributions.jl package](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Normal)

In [36]:
decode_distribution_model = Dict{Int,Normal}()
for s ∈ states

    # what indexes correspond to state s
    index_collection_state_s = findall(x-> x == s, encoded_in_sample);
    tmp = Array{Float64,1}();
    for i ∈ index_collection_state_s
        decoded_value = Rᵢ[i];
        push!(tmp, decoded_value);
    end
    decode_distribution_model[s] = fit_mle(Normal,tmp);
end
decode_distribution_model;

Then, generate a random value for the excess return by sampling the appropriate [Normal distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Normal). We develop `number_of_paths` trajectories, each containing `number_of_steps` values. We store these values in the `decoded_archive` variable:

In [37]:
in_sample_decoded_archive = Array{Float64,2}(undef, number_of_steps, number_of_paths);
for i ∈ 1:number_of_paths
    for j ∈ 1:number_of_steps
        s = encoded_archive[j,i];
        in_sample_decoded_archive[j,i] =  decode_distribution_model[s] |> d -> rand(d)
    end
end
in_sample_decoded_archive # actual excess growth value

87191×100 Matrix{Float64}:
 -0.965001    1.53357    0.0897358  …   0.173858   0.0496667   0.0608759
  0.108595    0.957741   0.0771891      0.236487   0.0422551   0.0367049
  0.843499   -0.798026   0.082507       0.100941   0.0825502   0.0504686
 -0.506217   -0.0313737  0.0984252      0.0956638  0.151508    0.134673
 -0.458532    0.225968   0.0862371      0.100993   0.234964    0.21137
  0.0169789   0.015718   0.116981   …   0.171387   0.128356    0.260906
  0.0355926   0.0314863  0.22672        0.330594   0.0874668   0.319636
  0.0646796   0.056789   0.236306       0.196844   0.148768    0.276305
  0.0595994   0.0261702  0.103892       0.245509   0.168632    0.132054
  0.0447115   0.0420291  0.11129        0.273774   0.140496    0.1553
  0.0909499   0.0449564  0.0458407  …   0.175211   0.0826818   0.220307
  0.169898    0.0492975  0.0221524      0.149072   0.0495035   0.213366
  0.23418     0.0733133  0.0332721      0.114837   0.046395    0.16384
  ⋮                                 ⋱ 

### Visualize simulated excess growth rate distribution

In [42]:
let
    q = plot(); 
    density!(in_sample_decoded_archive[:,1], lw=2, c=:deepskyblue1, label="Simulated", 
        bg="gray95", background_color_outside="white", framestyle = :box, fg_legend = :transparent, 
        xlims=(bounds[1,2], bounds[end,1]))
    for i ∈ 2:number_of_paths
        density!(in_sample_decoded_archive[:,i], lw=1, c=:deepskyblue1, label="")
    end
    density!(in_sample_dataset, c=:red, lw=3, label="Observed")
    xlabel!("Excess Annual Growth Rate $(ticker) (1/year)", fontsize=18)
    ylabel!("Probability Density (AU)", fontsize=18)
    current()
end

### Check: Are the predicted and observed return distributions the same? 
If our Markov model is correct, then the observed excess growth rate distribution and the excess growth distribution calculated by our model should look like they are drawn from the same distribution. To check this hypothesis, use [the ApproximateTwoSampleKSTest exported by the HypothesisTests.jl package](https://github.com/JuliaStats/HypothesisTests.jl) 
* `H0:null hypothesis` is that `x` and `y` are drawn from the same distribution against the `H1:alternative hypothesis` that `x` and `y` come from different distributions.

We have `number_of_paths` example trajectories, so let's do the test on each sample and compute an overall expected score. Specify a `pvalue_cutoff` value to check against. If the test returns `pvalue > pvalue_cutoff,` then we fail to reject `H0:null hypothesis`, i.e., `x` and `y` appear to be drawn from the same distribution.

In [39]:
pvalue_cutoff = 0.01; # 99% cutoff
pass_counter = 0;
for i ∈ 1:number_of_paths
    test_value = ApproximateTwoSampleKSTest(in_sample_dataset,in_sample_decoded_archive[:,i]) |> pvalue
    if (test_value > pvalue_cutoff)
        pass_counter += 1 # we pass (fail to reject) x and y are from the same distribution
    end
end
println("Pass percentage: $((pass_counter/number_of_paths)*100)%")

Pass percentage: 0.0%


## Task 3: Save the HMM model and other data to a file
In the project and advanced example for this module, we'll use the hidden Markov Model (HMM) we developed here. Let's save the model to disk to save some time later. 
* Use the [save(...) method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl.git) to write the model in a binary format. First, we specify a `path` in the `path_to_save_file` variable: 

In [40]:
# path_to_save_file = joinpath(_PATH_TO_DATA,"HMM-$(ticker)-1-min-aggregate.jld2");

Then we write an `HDF5 binary file` holding our data to the location specified by `path_to_save_file.` We use the [save(...) function exported by the JLD2.jl package to write a binary save file](https://github.com/JuliaIO/JLD2.jl.git) (later we'll use to the `load(...)` function to reload this data):

In [41]:
# save(path_to_save_file, Dict("model"=>model, 
#         "decode"=>decode_distribution_model, "stationary"=>π̄, 
#         "insampledataset"=>in_sample_dataset, "outofsampledataset"=>out_of_sample_dataset));

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.